# Building an Expected Goals Model

In this notebook, we will create a model for computing the probability of a shot being a goal. This probability is referred to as *Expected Goals* (xG) and it is a popular metric in football today to understand how good a team is at creating chances.

Based on the videos by [Prof. David Sumpter](https://uppsala.instructure.com/courses/28112/pages/2-statistical-models-of-actions), we will fit a *Logistic Regression* model to estimate xG. This model will have two input variables - distance of a shot from goal and angle of the shot to the width of the goal. To fit the model, we will use [event data](https://github.com/statsbomb/open-data) from La Liga (Spanish league) matches provided by Statsbomb.

## Imports

To load and inspect this data, we will need the `json` and `pandas` packages. We will need `numpy` for intermediate transformations and `statsmodels` for fitting the Logistic Regression model.

In [8]:
import json
import os
import pickle
from typing import List, Optional

import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from mplsoccer import Standardizer
from sklearn.metrics import mean_absolute_error

## Data

We have already downloaded the data and placed it in `../data/statsbomb/data` directory. Event data is available per match in a JSON file. So, in order to fetch data for all available La Liga matches, we first need to get the competition ID of La Liga. Let us do so by loading `competitions.json` and inspecting it.

In [9]:
competitions_fn: str = "../data/statsbomb/data/competitions.json"
with open(competitions_fn, "r") as f:
    competitions: pd.DataFrame = pd.read_json(f)
competitions

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,16,4,Europe,Champions League,male,False,False,2018/2019,2022-08-14T16:57:15.866765,2021-06-13T16:17:31.694,None,2022-08-14T16:57:15.866765
1,16,1,Europe,Champions League,male,False,False,2017/2018,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2021-01-23T21:55:30.425330
2,16,2,Europe,Champions League,male,False,False,2016/2017,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
3,16,27,Europe,Champions League,male,False,False,2015/2016,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
4,16,26,Europe,Champions League,male,False,False,2014/2015,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
5,16,25,Europe,Champions League,male,False,False,2013/2014,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
6,16,24,Europe,Champions League,male,False,False,2012/2013,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2021-07-10T13:41:45.751
7,16,23,Europe,Champions League,male,False,False,2011/2012,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
8,16,22,Europe,Champions League,male,False,False,2010/2011,2022-01-26T21:07:11.033473,2021-06-13T16:17:31.694,None,2022-01-26T21:07:11.033473
9,16,21,Europe,Champions League,male,False,False,2009/2010,2022-02-12T16:13:49.294747,2021-06-13T16:17:31.694,None,2022-02-12T16:13:49.294747


We see that La Liga has the ID `11`. Let us now load the list of La Liga matches for which event data is available. In `./data/statsbomb/data/matches/11/`, we have one JSON file for every La Liga season. Each of the JSON files provides basic information about all matches in that season. We are interested in getting the ID of each match so that we can load event data for that match from `./data/statsbomb/data/events`. So, let us write function to get IDs of all matches for which we want to get event data.

In [10]:
def get_competition_match_ids(comp_id: int, data_dir: str = "../data/statsbomb/data/") -> list:
    """
    Get IDs of matches from all seasons of a competition e.g. La Liga.
    :param comp_id:  Competition ID from one of those provided in `competitions.json`
    :param data_dir: Path to directory containing `competitions.json` and `matches` directory
    from Statsbomb open data.
    :return: List of match IDs for all seasons.
    """
    comp_dir: str = os.path.join(data_dir, "matches", str(comp_id))
    season_fns: list = os.listdir(comp_dir)
    season_fn: str
    match: dict
    match_ids: list = []
    for season_fn in season_fns:
        with open(os.path.join(comp_dir, season_fn), "r") as jf:
            matches: list = json.load(jf)

        for match in matches:
            match_ids.append(match["match_id"])

    return match_ids

In [11]:
required_match_ids: list = get_competition_match_ids(11)

Let us write a quick test for our function. Let us pick one match ID at random from the following files:
- `1.json`
- `27.json`
- `42.json`

and verify that our list contains these IDs.

In [12]:
assert 9609 in required_match_ids
assert 266166 in required_match_ids
assert 303532 in required_match_ids

Having fetched the match IDs, let us now incrementally load events from them. For our first model, let us only consider shots to goal from open play. To do so, we use our learnings from the [previous exploratory notebook](./00_loading_investigating_world_cup_data.ipynb).

In [13]:
def load_match_events(match_id: int, data_dir: str = "../data/statsbomb/data/events") -> List[dict]:
    """
    Load event data of a match.
    :param match_id: ID of the match which matches the name of the JSON file from which
    to load events
    :param data_dir: Path to the `events` directory of Statsbomb open data.
    :return: A list of dictionaries with each dictionary denoting an event i.e. on-ball action.
    """
    with open(os.path.join(data_dir, f"{match_id}.json"), "r") as jf:
        return json.load(jf)

def frame_events(e: list, match_id: int) -> pd.DataFrame:
    """
    Convert a list of on-ball match events to a Pandas dataframe.
    :param e:        List of dictionaries with each dictionary denoting an event.
    :param match_id: ID of the match whose events are transformed.
    :return: Pandas dataframe of events.
    """
    return (pd.json_normalize(e, sep="_")
            .assign(match_id=match_id))

def filter_events(e: pd.DataFrame, event_type: str, event_type_filter: Optional[dict] = None) -> pd.DataFrame:
    """
    Filter events to include the specified actions. Supported event types are
    one of `["Shot", "Pass"]`. Further filters are supplied as key-value pairs
    with the key of the dictionary interpreted as the column name and the
    dictionary value as the value in the column that will be searched for.
    :param e:                 Dataframe of on-ball match events.
    :param event_type:        Type of events to filter out. Supported values are one
    of `["Shot", "Pass"]`.
    :param event_type_filter: A dictionary specifying filters specific to the event
    specified.
    :return: A Pandas dataframe of events of the specified type.
    """
    if not event_type_filter:
        event_type_filter = {}

    required_event: pd.DataFrame = e.loc[e["type_name"] == event_type].set_index("id")
    col: str
    val: str
    for col, val in event_type_filter.items():
        required_event = required_event.loc[required_event[col] == val]

    return required_event

m_id: int
match_wise_shots: list = []
for m_id in required_match_ids:
    match_events: list = load_match_events(m_id)
    events: pd.DataFrame = frame_events(match_events, m_id)
    match_shots: pd.DataFrame = filter_events(events, "Shot", {"shot_type_name": "Open Play"})
    match_wise_shots.append(match_shots)

la_liga_shots: pd.DataFrame = pd.concat(match_wise_shots)
la_liga_shots

,index,period,timestamp,minute,second,possession,duration,type_id,type_name,possession_team_id,...,goalkeeper_punched_out,shot_follows_dribble,goalkeeper_success_out,shot_redirect,half_end_early_video_end,goalkeeper_lost_in_play,player_off_permanent,goalkeeper_saved_to_post,goalkeeper_success_in_play,pass_backheel
id,,,,,,,,,,,,,,,,,,,,,
d0a3a318-9e65-4f79-9a78-1e65d4ea5b82,104,1,00:01:56.139,1,56,7,1.483300,16,Shot,217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
572ebe7b-7efb-4370-98d4-3d178ce0c59d,247,1,00:05:25.322,5,25,15,0.446500,16,Shot,217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9f17a2a7-976c-4eb7-b4be-7cb6f2b9bdc1,311,1,00:06:29.899,6,29,19,1.162000,16,Shot,217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bb3d8291-0dba-40a9-9ed0-f395eef891f6,662,1,00:15:56.988,15,56,34,0.616861,16,Shot,217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71faa4ab-6734-4fa1-b309-29df1020285c,684,1,00:16:30.980,16,30,37,0.174000,16,Shot,217,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6653337b-5599-453f-872d-379cfb5823eb,3664,2,00:34:48.656,79,48,200,1.087292,16,Shot,215,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7afa362d-de22-4a37-ac93-7d2f7579c593,3730,2,00:36:38.351,81,38,204,0.293700,16,Shot,215,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36ec2b3a-e40c-40bb-8c60-fc92bebf3a93,3885,2,00:40:02.956,85,2,210,0.549300,16,Shot,215,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Baseline model

Let us now construct a baseline Logistic Regression model. To do so, we require the following information:
- X and Y coordinates from where the shot was taken
- Whether the shot resulted in a goal

From the X and Y coordinates, we will next create two more columns - one to compute the distance of the shot from goal and the other to compute the angle of the shot to the goal.

Looking at the data above, we see a column named `shot_statsbomb_xg`. We can use this column as a reference for our model results, but we won't consider those values to be the ground truth.

The X and Y coordinates are available as a list in the `location` column. We will need to extract them and put them in separate columns. The column `shot_outcome_id` tells us the outcome of the shot. Based on page 20 of the document `./data/statsbomb/doc/Open Data Events v4.0.0.pdf`, we can see that shots with `shot_outcome_id = 97` are goals while the others are not. So, we need to construct a boolean column from it accordingly. Let us define a function that will perform these steps.

In addition, the [tutorial](https://www.youtube.com/watch?v=wHOgINJ5g54) we follow for building this model uses Wyscout data and the two data providers measure the pitch and thus record shot coordinates in different units. As we will use the logic used for Wyscout data to compute shot distance and angle, we make use of the `Standardizer` class of `mplsoccer` package to transform our coordinates from Statsbomb to Wyscout units.

In [14]:
def create_shot_modelling_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Transform event data of shots for modelling. This includes:
    - Splitting the `location` column into two - one for representing X coordinate and the other Y.
    - Converting the coordinate values to Wyscout units as further computations assume
    the coordinates to be in Wyscout units.
    - Creating a boolean column representing if the shot resulted in a goal (1) or not (0).
    - Dropping all column except the coordinates, boolean indicator of goal, and Statsbomb's xG value.
    - Renaming Statsbomb's xG value column.
    :param df: Pandas dataframe of event data for shots.
    :return: A Pandas dataframe with columns - X and Y coordinate of the shot, boolean indicator of goal,
    and Statsbomb's xG.
    """
    statsbomb_to_wyscout = Standardizer(pitch_from="statsbomb", pitch_to="wyscout")
    return (df.assign(X=lambda x: [l[0] for l in x["location"]])
            .assign(Y=lambda x: [l[1] for l in x["location"]])
            .assign(X=lambda x: [round(statsbomb_to_wyscout.transform([xi], [yi])[0][0], 2)
                                 for xi, yi in zip(x["X"], x["Y"])])
            .assign(Y=lambda x: [round(statsbomb_to_wyscout.transform([xi], [yi])[1][0], 2)
                                 for xi, yi in zip(x["X"], x["Y"])])
            .assign(is_goal=lambda x: (x["shot_outcome_id"] == 97).astype(int))
            .filter(["X", "Y", "is_goal", "shot_statsbomb_xg"], axis=1)
            .rename(columns={"shot_statsbomb_xg": "statsbomb_xg"}))

la_liga_shots_model_data: pd.DataFrame = create_shot_modelling_data(la_liga_shots)
la_liga_shots_model_data

,X,Y,is_goal,statsbomb_xg
id,,,,
d0a3a318-9e65-4f79-9a78-1e65d4ea5b82,69.02,35.80,0,0.010060
572ebe7b-7efb-4370-98d4-3d178ce0c59d,91.60,69.30,0,0.062470
9f17a2a7-976c-4eb7-b4be-7cb6f2b9bdc1,76.88,28.15,0,0.012832
bb3d8291-0dba-40a9-9ed0-f395eef891f6,90.07,67.65,0,0.043368
71faa4ab-6734-4fa1-b309-29df1020285c,82.95,44.90,0,0.144784
...,...,...,...,...
6653337b-5599-453f-872d-379cfb5823eb,79.06,57.40,0,0.023162
7afa362d-de22-4a37-ac93-7d2f7579c593,90.93,33.70,0,0.110399
36ec2b3a-e40c-40bb-8c60-fc92bebf3a93,83.11,23.05,0,0.012569


Let us now compute shot distance and angle.

In [15]:
def transform_coordinates_for_computation(x: float, y: float) -> (float, float):
    """
    Question: What does this transformation do?
    :param x: X coordinate of the shot taken.
    :param y: Y coordinate of the shot taken
    :return: Tuple of transformed X, Y coordinates.
    """
    m_x: float = 100 - x
    c: float = abs(y - 50)

    return (m_x * 105) / 100, (c * 65) / 100

def compute_shot_distance(x: float, y: float) -> float:
    """
    Computes distance of a shot from goal.
    :param x: X coordinate of the shot taken.
    :param y: Y coordinate of the shot taken.
    :return: Float denoting distance (in meters) of the shot from goal.
    """
    t_x: float
    t_y: float
    t_x, t_y = transform_coordinates_for_computation(x, y)

    return np.sqrt(t_x ** 2 + t_y ** 2)

def compute_shot_angle(x: float, y: float) -> float:
    """
    Computes the angle of the shot to the width of the goal post.
    :param x: X coordinate of the shot taken.
    :param y: Y coordinate of the shot taken
    :return: Float denoting the angle (in radians) of the shot.
    """
    t_x: float
    t_y: float
    t_x, t_y = transform_coordinates_for_computation(x, y)

    angle: float = np.arctan((7.32 * t_x) / (t_x ** 2 + t_y ** 2 - (7.32 / 2) ** 2))
    if angle < 0:
        angle = np.pi + angle

    return angle

la_liga_xg_model_data: pd.DataFrame = (la_liga_shots_model_data
                                       .assign(dist=lambda x: [compute_shot_distance(xc, yc)
                                                               for xc, yc in zip(x["X"], x["Y"])])
                                       .assign(angle=lambda x: [compute_shot_angle(xc, yc)
                                                                for xc, yc in zip(x["X"], x["Y"])]))
la_liga_xg_model_data

,X,Y,is_goal,statsbomb_xg,dist,angle
id,,,,,,
d0a3a318-9e65-4f79-9a78-1e65d4ea5b82,69.02,35.80,0,0.010060,33.813145,0.207693
572ebe7b-7efb-4370-98d4-3d178ce0c59d,91.60,69.30,0,0.062470,15.335235,0.283289
9f17a2a7-976c-4eb7-b4be-7cb6f2b9bdc1,76.88,28.15,0,0.012832,28.125348,0.224655
bb3d8291-0dba-40a9-9ed0-f395eef891f6,90.07,67.65,0,0.043368,15.502586,0.324434
71faa4ab-6734-4fa1-b309-29df1020285c,82.95,44.90,0,0.144784,18.206832,0.390787
...,...,...,...,...,...,...
6653337b-5599-453f-872d-379cfb5823eb,79.06,57.40,0,0.023162,22.506983,0.315451
7afa362d-de22-4a37-ac93-7d2f7579c593,90.93,33.70,0,0.110399,14.246090,0.352413
36ec2b3a-e40c-40bb-8c60-fc92bebf3a93,83.11,23.05,0,0.012569,24.927401,0.210362


Let us now fit a Logistic Regression model using statsmodel's `glm()` method. As our output is binary, we specify the `family` argument to be `sm.families.Binomial()`. After fitting the model, let us print a summary of the model.

In [16]:
baseline_model = smf.glm(formula="is_goal ~ dist + angle", data=la_liga_xg_model_data,
                         family=sm.families.Binomial()).fit()
baseline_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                is_goal   No. Observations:                11700
Model:                            GLM   Df Residuals:                    11697
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -4024.5
Date:                Sat, 10 Dec 2022   Deviance:                       8049.0
Time:                        22:33:31   Pearson chi2:                 1.15e+04
No. Iterations:                     6   Pseudo R-squ. (CS):            0.08902
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.2611      0.187     -6.739      0.000      -1.628      -0.894
dist          -0.0936      0.008    -11.455      0.000      -0.110      -0.078
angle          1.4011      0.162      8.636      0.000       1.083       1.719
==============================================================================
"""

Looking at the model summary above, in particular the last table, we see that the probability of a shot becoming a goal decreases with increasing distance. We say this based on the negative value of the coefficient. Similarly, as the shot angle increases, i.e. as the shot is taken from between the goal posts, the probability of hitting the back of the net also increases. The near-zero P-values of both the coefficients suggest that we have sufficient evidence to reject the null hypothesis that the true coefficient value is zero.

Let us now save the model parameters in a `.pkl` file.

In [17]:
def save_model(params: pd.Series, model_fn: str, model_dir: str = "../models"):
    """
    Save parameters of Logistic Regression to a pickle file.
    :param params:    A Pandas Series of Logistic Regression parameters.
    :param model_fn:  Name of pickle file (with extension) to save the parameters to.
    :param model_dir: Directory to save the model to.
    """
    with open(os.path.join(model_dir, model_fn), "wb") as pf:
        pickle.dump(params, pf)

baseline_model_params: pd.Series = baseline_model.params
baseline_model_fn: str = "baseline_logistic_model.pkl"
save_model(baseline_model_params, baseline_model_fn)

## Serving the model

Let us now define a function that loads the parameters from a pickle file and computes the xG value.

In [18]:
def compute_xg(xc: float, yc: float, model_fn: str, model_dir: str = "../models") -> float:
    """
    Compute the xG of a shot given the (X, Y) coordinates of where it was taken from.
    :param xc:        X-coordinate of where the shot was taken from.
    :param yc:        Y-coordinate of where the shot was taken from.
    :param model_fn:  Name of the file containing Logistic Regression parameters.
    :param model_dir: Directory containing the model file.
    :return: A float representing xG value.
    """
    shot_dist: float = compute_shot_distance(xc, yc)
    shot_ang: float = compute_shot_angle(xc, yc)

    with open(os.path.join(model_dir, model_fn), "rb") as pf:
        model_params: pd.Series = pickle.load(pf)

    linear_sum: float = (model_params["Intercept"]
                         + (shot_dist * model_params["dist"])
                         + (shot_ang * model_params["angle"]))

    return 1 / (1 + np.exp(-1 * linear_sum))

la_liga_xg_model_data = (la_liga_xg_model_data
                         .assign(xg=lambda x: [compute_xg(xi, yi, baseline_model_fn) for xi, yi in zip(x["X"], x["Y"])]))
la_liga_xg_model_data

,X,Y,is_goal,statsbomb_xg,dist,angle,xg
id,,,,,,,
d0a3a318-9e65-4f79-9a78-1e65d4ea5b82,69.02,35.80,0,0.010060,33.813145,0.207693,0.015755
572ebe7b-7efb-4370-98d4-3d178ce0c59d,91.60,69.30,0,0.062470,15.335235,0.283289,0.091168
9f17a2a7-976c-4eb7-b4be-7cb6f2b9bdc1,76.88,28.15,0,0.012832,28.125348,0.224655,0.027156
bb3d8291-0dba-40a9-9ed0-f395eef891f6,90.07,67.65,0,0.043368,15.502586,0.324434,0.094707
71faa4ab-6734-4fa1-b309-29df1020285c,82.95,44.90,0,0.144784,18.206832,0.390787,0.081841
...,...,...,...,...,...,...,...
6653337b-5599-453f-872d-379cfb5823eb,79.06,57.40,0,0.023162,22.506983,0.315451,0.050903
7afa362d-de22-4a37-ac93-7d2f7579c593,90.93,33.70,0,0.110399,14.246090,0.352413,0.109032
36ec2b3a-e40c-40bb-8c60-fc92bebf3a93,83.11,23.05,0,0.012569,24.927401,0.210362,0.035593


From the dataframe above, we see that our computed xG differs from the Statsbomb one. If we were to measure the difference in absolute terms, we see that the two xG values differ by about 7%.

In [19]:
mean_absolute_error(la_liga_xg_model_data["statsbomb_xg"], la_liga_xg_model_data["xg"])

0.071261113252593

## Improvements

If we look at the documentation of event information in `./data/statsbomb/doc/Open Data Events v4.0.0.pdf`, we can spot additional parameters that might improve the model. These include:
- Freeze-frame which tells us about the opposition players in the vicinity when a shot was taken. This can be an important variable as more players around means more pressure on the player which in turn can lead to a false shot.
- Open goal which tells us if the shot was taken in front of an open goal.
- Deflected which tells us if the shot was deflected.
- Technique whose values can be one of Backheel, Diving header, Half volley, Lob, Normal, Overhead kick, or volley.
- Body part which indicates if the shot was taken with the head, left foot, right foot, or other body part. This variable combined with information about which foot a player prefers can be useful in predicting if a shot will turn into a goal.

In his video [The Ultimate Guide to Expected Goals](https://www.youtube.com/watch?v=310_eW0hUqQ), Prof. Sumpter asserts that given the right variables, models more complex than Logistic Regression might not provide much better performance. Thus, improvements to the model can focus on adding more variables first and evaluating if they contribute to the model fit before experimenting with other models.